## Hierarchical Forecasting with Different level's of Data

In [17]:
import pandas as pd
import sys
sys.path.insert(0, '../')

import constants as const
import src.utils as util
import src.calculate_errors as err

import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [2]:
def calculate_grid_error(model_path):
    data = pd.read_csv(f'../ts_data/grid.csv', index_col=[0])
    look_back = 14 * 7  # 14 hours in to 7 days

    # train, val, test split
    train, val, test = util.split_hourly_data(data, look_back)
    train_df = train[['power']]
    
    results_df = pd.read_csv(f'../{model_path}/grid.csv', index_col=[0])
    test_sample = results_df['power'].values
    forecasts = results_df['fc'].values
    horizon = 14
    
    mean_err, error_dist = err.test_errors_nrmse(train_df.values, test_sample, forecasts, horizon)
    return mean_err, error_dist

In [18]:
def read_loss_curve(path):
    return pd.read_pickle(f'../{path}/training_loss_grid_iteration')

def plot_loss(loss_array, models):

    fig = make_subplots(rows = len(loss_array), shared_xaxes=True)
    
    i =0
    for loss in loss_array:
        fig.add_trace(go.Scatter(y= loss['loss'] , name = f'{models[i]}_train-loss' ), row = i+1, col=1)
        fig.add_trace(go.Scatter(y= loss['val_loss'], name = f'{models[i]}_val-loss'), row = i+1, col=1)
        i +=1
    fig.show()
    
def show_loss(path_list, model_names):
    loss_vals = []
    for path in path_list:
        loss_vals.append(read_loss_curve(path))
    plot_loss(loss_vals, model_names)

## Model 1 - Architecture

<img src="../images/model1.png">

In [11]:
path1 = 'combined_nn_results/new_models/model3'
rmse1, dist1 = calculate_grid_error(path1)

In [12]:
rmse1

0.6572463576578557

## Model 2 - Architecture

<img src="../images/model2.png">

In [14]:
path2 = 'combined_nn_results/new_models/model4'
rmse2, dist2 = calculate_grid_error(path2)

In [15]:
rmse2

0.643096926566697

## Model 3 - Architecture

<img src="../images/model3.png">

In [22]:
# dense layer 14 output from each branch
path3 = 'combined_nn_results/new_models/model5'
rmse3, dist3 = calculate_grid_error(path3)

In [23]:
rmse3

0.6145534915204655

In [25]:
# dense layer 40 output from each branch
path3_1 = 'combined_nn_results/new_models/model6'
rmse3_1, dist3_1 = calculate_grid_error(path3_1)

In [26]:
rmse3_1

0.6308694408841671

## Model 4 - Architecture 

<img src="../images/model4.png">

## Loss Curves

In [27]:
show_loss([path1, path2, path3, path3_1], ['grid+pc', 'pc', 'grid+pc embedding', 'grid+pc embedding2'])